LABORATORIO 1 - INTELIGENCIA ARTIFICIAL
Diego García 22404
César López 22838

# TASK 1  
 ### - PREGUNTA 1
Considera un modelo de regresión lineal con dos características, X₁ y X₂, y sus pesos correspondientes w₁ y w₂. 
Si el modelo predice una salida y mediante la ecuación y = 2w₁X₁ + 3w₂X₂ + 1, ¿cuál es la interpretación del coeficiente 3w₂ en el contexto del modelo?

- RESPUESTA:
3w₂ representa el cambio que recibe y cuando X₂ va cambiando sus valores. Esto significa que mientras mayor sea el valor de 3w₂ mayor será el valor final de y con cada valor distinto de X₂.


### - PREGUNTA 2  
Explica el concepto de multicolinealidad en el contexto de la regresión lineal. ¿Cómo afecta la
multicolinealidad a la interpretación de los coeficientes de regresión individuales?

- RESPUESTA:
Es cuando dos o mas variables predictoras estan altamente correlacionadas entre si, lo que implica que existe una redundancia en la informacion analizada, de donde se obtienen estas variables. Esto puede llegar a afectar aumentando el cesgo de las predicciones que realiza el modelo fuera del contexto con el que fue entrenado. Tambien el modelo puede llegar a confundir en cual es la variable que influye en la otra.

TASK 2 - CLASIFICACIÓN DE SITIOS DE PHISHING

Descargar el dataset de: https://www.kaggle.com/datasets/shashwatwork/web-page-phishing-detection-dataset 


In [1]:
#Librerías para cargar los datos

import pandas as pd
import numpy as np

In [ ]:
dataset = 

# TASK 2 - 2.2 - NO LIBRERIAS

In [76]:
import pandas as pd
import numpy as np
import altair as alt
import matplotlib.pyplot as plt
import plotly.express as px


In [77]:
def prepare_file(dataset_path: str) -> pd.DataFrame:
    dataset = pd.read_csv(dataset_path)
    dataset["status"] = dataset["status"].map({"phishing": 1, "legitimate": 0})
    return dataset.drop("url", axis=1)


In [78]:
def split_dataset(dataset: pd.DataFrame, frac: float = 0.8, random_state: int = 42):
    train = dataset.sample(frac=frac, random_state=random_state)
    test = dataset.drop(train.index)
    return train, test



In [79]:
def KNN(
    x_train: pd.DataFrame, y_train: pd.Series, x_test: pd.DataFrame, vecinosCant: int
) -> list:
    y_pred = []
    x_train = x_train.to_numpy()
    x_test = x_test.to_numpy()
    for x in x_test:
        distances = np.sqrt(((x_train - x) ** 2).sum(axis=1))
        k_nearest = np.argpartition(distances, vecinosCant)[:vecinosCant]
        k_nearest_labels = y_train.iloc[k_nearest]
        y_pred.append(k_nearest_labels.value_counts().idxmax())
    return y_pred


In [80]:
def Metrics(test_y: np.ndarray, y_pred: np.ndarray) -> dict:
    TP = np.sum((test_y == 1) & (y_pred == 1))
    FP = np.sum((test_y == 0) & (y_pred == 1))

    precision = TP / (TP + FP) if (TP + FP) > 0 else 0

    return {"precision": precision}


In [81]:
def Graphic(test_x, test_y, y_pred):
    results_df = test_x.copy()
    results_df["Real"] = test_y
    results_df["Predicho"] = y_pred
    results_df["Correcto"] = results_df["Real"] == results_df["Predicho"]

    fig = px.scatter(
        results_df,
        x="length_url",
        y="length_hostname",
        color="Correcto",
        symbol="Real",
        color_discrete_map={True: "green", False: "red"},
        title="Clasificación de KNN con Plotly",
        labels={
            "length_url": "Longitud del URL",
            "length_hostname": "Longitud del Hostname",
        },
    )

    fig.update_traces(marker=dict(size=12))
    fig.show()



In [ ]:
dataset_path = "./dataset_phishing.csv"
dataset = prepare_file(dataset_path)

train, test = split_dataset(dataset)

train_y = train.pop("status")
train_x = train
test_y = test.pop("status")
test_x = test

prediction = KNN(train_x, train_y, test_x, 3)

metrics = Metrics(test_y.to_numpy(), np.array(prediction))
print(f"Precision metric: {metrics['precision']:.2f}")

Graphic(test_x, test_y.to_numpy(), np.array(prediction))

# TASK 2.2 - CON LIBRERIA

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report
import pandas as pd

In [2]:
def prepare_file_sklearn(dataset_path: str) -> pd.DataFrame:
    dataset = pd.read_csv(dataset_path)
    dataset["status"] = dataset["status"].map({"phishing": 1, "legitimate": 0})
    dataset = dataset.drop("url", axis=1)
    return dataset

In [3]:
dataset_path = "./dataset_phishing.csv"
dataset = prepare_file_sklearn(dataset_path)

X = dataset.drop("status", axis=1)
y = dataset["status"]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [4]:

k = 3
knn_model = KNeighborsClassifier(n_neighbors=k)
knn_model.fit(X_train, y_train)

y_pred = knn_model.predict(X_test)


In [5]:

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Resultados con Scikit-learn:")
print(f"Accuracy (Exactitud): {accuracy:.2f}")
print(f"Precision (Precisión): {precision:.2f}")
print(f"Recall (Sensibilidad): {recall:.2f}")
print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred))

Resultados con Scikit-learn:
Accuracy (Exactitud): 0.84
Precision (Precisión): 0.82
Recall (Sensibilidad): 0.86

Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.85      0.82      0.84      1157
           1       0.82      0.86      0.84      1129

    accuracy                           0.84      2286
   macro avg       0.84      0.84      0.84      2286
weighted avg       0.84      0.84      0.84      2286



### Cual implementacion fue mejor? Por que? 

- Usando ambas formas para entrenar el modelo, nos dimos cuenta que la implementación con Scikit-learn fue mejor. Fue mucho más rápida y sencilla de escribir, ya que la librería se encarga de casi todo el trabajo pesado (tomando en cuenta que despues de haberlo intentado hacer *from scratch* me hizo pensar mas facil en la estructura del codigo ya con la libreria). Scikit-learn es más práctica y confiable para obtener resultados precisos y en menos tiempo. Además, facilita el análisis y la comparación de resultados. Al ser la sintaxis mas amigable y literal, ayuda bastante relacionar los proceso del algoritmo como algo toerico a la implementacion en el desarrollo de un programa. Algo bueno de haberlo hecho paso a paso, o sin librerias, es que me ayudo a aclarar un poco las dudas que tenia sobre el algoritmo.